### Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.analyses import detect_pbe_epochs
import pandas as pd
from scipy import stats
import seaborn as sns
from neuropy.plotting import Fig

import subjects

### Noisy pbes by peak count

In [ ]:
from neuropy.core import Epoch
from scipy.ndimage import gaussian_filter1d

# sessions = subjects.nsd.mua_sess+ subjects.sd.mua_sess
sessions = subjects.sd.ratKday1

for sub, sess in enumerate(sessions):
    pbe = sess.pbe
    pbe_df = pbe.to_dataframe()

    artifact_thresh = stats.zscore(pbe_df.peak_counts)
    bad_idx = artifact_thresh > 7
    pbe_df = pbe_df[~bad_idx]
    new_pbe = Epoch(pbe_df, metadata=pbe.metadata)

    # pbe.save(sess.filePrefix.with_suffix('.pbe'))

In [ ]:
_, ax = plt.subplots()

ax.plot(artifact_thresh)

### Pooled Pbe detection

In [ ]:
from neuropy.core import Epoch
from scipy.ndimage import gaussian_filter1d

# sessions = subjects.nsd.mua_sess+ subjects.sd.mua_sess
sessions = subjects.mua_sess()

for sub, sess in enumerate(sessions):
    # mua =sess.neurons.get_mua()
    # mua.save(sess.filePrefix.with_suffix(".mua.npy"))

    smth_mua = sess.mua.get_smoothed(sigma=0.02)
    pbe = detect_pbe_epochs(smth_mua, duration=(0.08, 0.5))
    pbe_df = pbe.to_dataframe()

    artifact_thresh = stats.zscore(pbe_df.peak_counts)
    bad_idx = artifact_thresh > 7
    pbe_df = pbe_df[~bad_idx]
    pbe = Epoch(pbe_df, metadata=pbe.metadata)

    pbe.save(sess.filePrefix.with_suffix(".pbe"))

In [ ]:
_, ax = plt.subplots()

ax.plot(stats.zscore(pbe.to_dataframe().peak_counts))

### Filter PBE based on the following criterias
- Ripple band power >=1 SD
- Number of cells firing (Using only stable pyramidal cells)
- Theta/delta ratio should not be high
- Number of bins having at least one spike

In [4]:
from neuropy.utils.signal_process import (
    filter_sig,
    hilbertfast,
    hilbert_ampltiude_stat,
    FourierSg,
)
from scipy.ndimage import gaussian_filter1d
from neuropy.core import Epoch

sessions = subjects.pf_sess()

sigma = 0.0125
dt = 1 / 1250
smooth = lambda x: gaussian_filter1d(x, sigma=sigma / dt, axis=-1)
smooth_spect = lambda x: gaussian_filter1d(x, sigma=1 / 0.2, axis=-1)

for s, sess in enumerate(sessions):
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
    fs = sess.eegfile.sampling_rate
    pbe_epochs = sess.pbe.flatten()
    starts = sess.pbe.starts
    stops = sess.pbe.stops

    # --------- Ripple power --------
    ripple_channels = sess.ripple.metadata["channels"]
    lfp = sess.eegfile.get_signal(ripple_channels)
    lfp_t = lfp.time

    signals = lfp.traces
    power = np.zeros(signals.shape[1])
    for sig in signals:
        yf = filter_sig.bandpass(sig, lf=125, hf=250, fs=fs)
        power += np.abs(hilbertfast(yf))

    power = smooth(power / signals.shape[0])

    if sess.artifact is not None:
        ignore_times = sess.artifact.as_array()
    else:
        ignore_times = None
    if ignore_times is not None:
        assert ignore_times.ndim == 2, "ignore_times should be 2 dimensional array"
        noisy_frames = np.concatenate(
            [
                (np.arange(start * fs, stop * fs)).astype(int)
                for (start, stop) in ignore_times
            ]
        )

        power[noisy_frames] = 0
    power = stats.zscore(power)

    pbe_rpl_power = stats.binned_statistic(
        lfp_t, power, statistic="max", bins=pbe_epochs
    )[0][::2]
    pbe_rpl_bool = pbe_rpl_power >= 1

    # --- Number of neurons should exceed atleast 5 neurons --------
    n_spikes = [np.histogram(_, bins=pbe_epochs)[0][::2] for _ in neurons.spiketrains]
    n_neurons_bool = (np.array(n_spikes) > 0).sum(axis=0) >= 5

    # ----- Percentage of bins that should have atleast one spike ----
    pbe_spikes, nbins = neurons.get_spikes_in_epochs(sess.pbe, bin_size=0.02)
    each_bin_bool = np.array([np.sum(arr.sum(axis=0) > 0) for arr in pbe_spikes])
    each_bin_bool = each_bin_bool / nbins > 0.8

    # ------ PBE should not happen during running -----
    pbe_peak = sess.pbe.to_dataframe().peak_time
    run_epochs = sess.maze_run.flatten()
    run_bool = np.digitize(pbe_peak, run_epochs) % 2 == 1

    # --- PBE should not have high theta (may be important as sleep deprivation has high theta )---------

    # good_channels = sess.recinfo.good_channels.astype("int")
    # maze = sess.paradigm["maze"].flatten()
    # maze_lfp = sess.eegfile.get_signal(good_channels, *maze)
    # theta_stat = hilbert_ampltiude_stat(
    #     maze_lfp.traces, freq_band=(5, 16), fs=fs, statistic="mean"
    # )
    # best_theta_chan = maze_lfp.channel_id[theta_stat.argmax()]

    # theta_signal = sess.eegfile.get_signal(best_theta_chan)
    # theta_spect = FourierSg(theta_signal, window=1, overlap=0.8)
    # theta_delta_ratio = stats.zscore(smooth_spect(theta_spect.theta_delta_ratio))
    # theta_delta_ratio = np.interp(lfp_t, theta_spect.time, theta_delta_ratio)

    # pbe_theta_power = stats.binned_statistic(
    #     lfp_t, theta_delta_ratio, statistic="max", bins=pbe_epochs
    # )[0][::2]
    # pbe_theta_bool = pbe_theta_power < 1

    df = pd.DataFrame(
        dict(
            start=starts,
            stop=stops,
            label="pbe",
            is_rpl=pbe_rpl_bool,
            is_5units=pbe_rpl_bool,
            is_80percentbins=each_bin_bool,
            is_rest=~run_bool,
            # is_lowtheta=pbe_theta_bool,
        )
    )
    Epoch(epochs=df).save(sess.filePrefix.with_suffix(".pbe.filters"))

#Sessions = 13
/data/Clustering/sessions/RatK/Day2/RatK_Day2_2019-08-08_04-00-00.pbe.filters saved
/data/Clustering/sessions/RatN/Day2/RatN_Day2_2019-10-11_03-58-54.pbe.filters saved
/data/Clustering/sessions/RatS/Day2NSD/RatS-Day2NSD-2020-11-27_10-22-29.pbe.filters saved
/data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.pbe.filters saved
/data/Clustering/sessions/RatV/RatVDay1NSD/RatV_Day1NSD_2021-10-02_08-10-23.pbe.filters saved
/data/Clustering/sessions/RatV/RatVDay3NSD/RatV_Day3NSD_2021-10-07_08-10-12.pbe.filters saved
/data/Clustering/sessions/RatK/Day1/RatK_Day1_2019-08-06_03-44-01.pbe.filters saved
/data/Clustering/sessions/RatN/Day1/RatN_Day1_2019-10-09_03-52-32.pbe.filters saved
/data/Clustering/sessions/RatS/Day3SD/RatS_Day3SD_2020-11-29_07-53-30.pbe.filters saved
/data/Clustering/sessions/RatU/RatUDay1SD/RatU_Day1SD_2021-07-22_07-55-46.pbe.filters saved
/data/Clustering/sessions/RatU/RatUDay4SD/RatU_Day4SD_2021-07-29_08-23-06.pbe.filters saved
/data

### Test PBE filters using neuroscope

In [ ]:
from neuropy.analyses.spkepochs import detect_pbe_epochs

sess = subjects.sd.ratUday1[0]
neurons = sess.neurons
sort_ind = neurons.firing_rate.argsort()
pbe_epochs = sess.pbe.flatten()
pbe_filter = sess.pbe_filters.to_dataframe()
good_bool = (
    pbe_filter.is_rpl
    # & pbe_filter.is_5neurons
    # & pbe_filter.is_lowtheta
    & pbe_filter.is_rest
)
# sess.recinfo.write_epochs(sess.pbe[good_bool.values],'.pbe')
sess.recinfo.write_epochs(sess.pbe, ".apb")
sess.recinfo.write_neurons(neurons[sort_ind])

# smth_mua = sess.mua.get_smoothed(sigma=0.02)
# new_pbe = detect_pbe_epochs(smth_mua,duration=(0.03,0.6))
# sess.recinfo.write_epochs(new_pbe,'.npb')

In [ ]:
len(sess.pbe)

In [ ]:
sess